In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test = pd.read_csv("../input/zimnat-insurance-recommendation-challenge/Test.csv")
submission = pd.read_csv("../input/zimnat-insurance-recommendation-challenge/SampleSubmission.csv")
train = pd.read_csv("../input/zimnat-insurance-recommendation-challenge/Train.csv")

In [ ]:
train

In [ ]:
set(train.columns) - set(test.columns)

In [ ]:
test

In [ ]:
dataset = pd.concat([train,test]).reset_index()
dataset

In [ ]:
dataset['join_date'] = pd.to_datetime(dataset['join_date'], format='%d/%m/%Y').dt.year
dataset['age'] = dataset['join_date'] - dataset['birth_year']
dataset

In [ ]:
dataset_final = dataset[['age','sex','marital_status','branch_code','occupation_code','occupation_category_code']]
dataset_final

In [ ]:
dataset_final['sex'].value_counts()

In [ ]:
dataset_final['marital_status'].value_counts()

In [ ]:
dataset_final['branch_code'].value_counts()

In [ ]:
dataset_final['occupation_category_code'].value_counts()

In [ ]:
train['occupation_code'].value_counts()

In [ ]:
len(train['occupation_code'].value_counts())

In [ ]:
cutoff = 100
df_col = dataset_final['occupation_code'].value_counts()[dataset_final['occupation_code'].value_counts() > cutoff]
df_col

In [ ]:
len(df_col.index),df_col.index

In [ ]:
dict_map = {value:key+1 for key,value in enumerate(df_col.index)}
dict_map['etc'] = 37 
dict_map

In [ ]:
df_error = dataset_final['occupation_code'].value_counts()[dataset_final['occupation_code'].value_counts() <= cutoff]
df_error

In [ ]:
dataset_final

In [ ]:
# occupation_code : cutoff below 100, rename as etc

dataset_final['occupation_code'].replace(to_replace = list(df_error.index), value = 'etc')
dataset_final['occupation_code']

dataset_final[dataset_final['occupation_code'] == 'Q57T']

In [ ]:
df_error.index

In [ ]:
for value in df_error.index:
    dataset_final.loc[dataset_final['occupation_code'] == value,'occupation_code'] = 'etc'

In [ ]:
dataset_final['occupation_code'] = dataset_final['occupation_code'].map(dict_map)
dataset_final

In [ ]:
from sklearn.preprocessing import LabelEncoder
dataset_final['sex'] = LabelEncoder().fit_transform(dataset_final['sex'])
dataset_final['marital_status'] = LabelEncoder().fit_transform(dataset_final['marital_status'])
dataset_final['branch_code'] = LabelEncoder().fit_transform(dataset_final['branch_code'])
dataset_final['occupation_category_code'] = LabelEncoder().fit_transform(dataset_final['occupation_category_code'])
dataset_final

In [ ]:
dataset.columns

In [ ]:
dataset

In [ ]:
dataset_final.iloc[:len(train)*8//10]

In [ ]:
def prediction_column(columns):
    x_train = dataset_final.iloc[:len(train)*8//10]
    x_val = dataset_final.iloc[len(train)*8//10:len(train)]
    x_test = dataset_final.iloc[len(train):]

    y_train = dataset[columns].iloc[:len(train)*8//10]
    y_val = dataset[columns].iloc[len(train)*8//10:len(train)]
    
    import time
    import xgboost as xgb
    ts = time.time()

    model = xgb.XGBClassifier(
        max_depth=10,
        n_estimators=1000,
        min_child_weight=0.5, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.1,
    #     tree_method='gpu_hist',
        seed=42)

    model.fit(
        x_train, 
        y_train, 
        eval_metric="rmse", 
        eval_set=[(x_train, y_train), (x_val, y_val)], 
        verbose=True, 
        early_stopping_rounds = 20)

    print(time.time() - ts)

    Y_test_class = model.predict(x_test)
    
    df = pd.DataFrame(data=Y_test_class, columns=["column1"])
    df['column1'].value_counts().sort_values().plot(kind = 'barh')
    
    return Y_test_class

In [ ]:
dataset.columns

In [ ]:
Y1 = prediction_column('RIBP')

In [ ]:
x_train = dataset_final.iloc[:len(train)*8//10]
x_val = dataset_final.iloc[len(train)*8//10:len(train)]
x_test = dataset_final.iloc[len(train):]

y_train = dataset['P5DA'].iloc[:len(train)*8//10]
y_val = dataset['P5DA'].iloc[len(train)*8//10:len(train)]

import time
import xgboost as xgb
ts = time.time()

model = xgb.XGBClassifier(
    max_depth=10,
    n_estimators=1000,
    min_child_weight=0.5, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,
#     tree_method='gpu_hist',
    seed=42)

model.fit(
    x_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(x_train, y_train), (x_val, y_val)], 
    verbose=True, 
    early_stopping_rounds = 20)

print(time.time() - ts)

Y_test_class = model.predict(x_test)
Y_test_class

In [ ]:
df = pd.DataFrame(data=Y_test_class, columns=["column1"])
df['column1'].value_counts().sort_values().plot(kind = 'barh')

In [ ]:
test

In [ ]:
Y1 = prediction_column('RIBP')

In [ ]:
submission_draft = pd.DataFrame()
submission_draft['index'] = test['ID']
submission_draft['P5DA'] = Y_test_class
submission_draft['RIBP'] = Y1
for column in dataset.columns[-20:-1]:
    submission_draft[column] = prediction_column(column)
submission_draft

In [ ]:
submission_draft

In [ ]:
submission_draft.index = submission_draft['index']
submission_draft = submission_draft.drop(['index'],axis=1)
submission_draft

In [ ]:
submission_draft

In [ ]:
output = []
for i in range(len(submission_draft)):
    output += list(submission_draft.iloc[0].values)
output

In [ ]:
submission_sample = submission.copy()
submission_sample['Label'] = output
submission_sample

In [ ]:
submission_sample.to_csv('submission_XGBClassifier.csv',index=False)

In [ ]:
submission_sample['Label'].value_counts().sort_values().plot(kind = 'barh')

In [ ]:
submission

spam

In [ ]:
dict_map

In [ ]:
for key in dict_map:
    print(key, dict_map[key])

In [ ]:
dataset_final['occupation_code'].value_counts()

In [ ]:
len(dataset_final['occupation_code'].value_counts().index),dataset_final['occupation_code'].value_counts().index

In [ ]:
dict_map

In [ ]:
list(dict_map.values())

In [ ]:
dict_map.values()

In [ ]:
dict_map

In [ ]:
df_error.index

In [ ]:
len(df_error), sum(df_error.values)

In [ ]:
df_col.plot(kind = 'barh')

In [ ]:
df_col

In [ ]:
df_error

In [ ]:
df_error.sort_values().plot(kind = 'barh')

In [ ]:
train['occupation_code'].value_counts()[train['occupation_code'].value_counts() == 5].value_counts().sort_values().plot(kind = 'barh')

In [ ]:
df_error.plot(kind = 'barh')

In [ ]:
df_error.value_counts().sort_values().plot(kind = 'barh')

In [ ]:
df_error.value_counts().sort_values().plot(kind = 'barh')

In [ ]:
df_error.value_counts().values

In [ ]:
sum(np.array(df_error.value_counts().index)*np.array(df_error.value_counts().values))

In [ ]:
df_error.value_counts()

In [ ]:
df_col.plot(kind = 'barh')

In [ ]:
df_col.value_counts().plot(kind = 'barh')

In [ ]:
df_col.value_counts().sort_values().plot(kind = 'barh')

In [ ]:
df_col.value_counts()

In [ ]:
test.columns

In [ ]:
test.columns[:8]

In [ ]:
test.columns[8:]

In [ ]:
len(test.columns[8:])

In [ ]:
submission

test['join_date'] = pd.to_datetime(test['join_date'])
test

test.columns

def non_object_type_detection(data):
    data_non_object_columns = data.dtypes[data.dtypes != object].index.values
    for columns in data_non_object_columns:
        print(columns)
        print(data[columns].value_counts())
        print()

def object_type_detection(data,detail=False):
    data_object_columns = data.dtypes[data.dtypes == object].index.values
    for columns in data_object_columns:
        if detail:
            print(columns)
            print(data[columns].value_counts())
            print()
    return data_object_columns

object_type_detection(train)

object_type_detection(train,detail=True)